# Task Overview

In this task, your goal is to verify the impact of data noise level in neural network training.
You should use MLP architecture trained on MNIST dataset (like in previous lab exercises).


We will experiment with two setups:
1. Pick X. Take X% of training examples and reassign their labels to random ones. Note that we don't change anything in the test set.
2. Pick X. During each training step, for each sample, change values of X% randomly selected pixels to random values. Note that we don't change anything in the test set.

For both setups, check the impact of various levels of noise (various values of X%) on model performance. Show plots comparing crossentropy (log-loss) and accuracy with varying X%, and also comparing two setups with each other.
Prepare short report briefly explaining the results and observed trends. Consider questions like "why accuracy/loss increases/decreases so quickly/slowly", "why Z is higher in setup 1/2" and any potentially surprising things you see on charts.

### Potential questions, clarifications
* Q: Can I still use sigmoid/MSE loss?
  * You should train your network with softmax and crossentropy loss (log-loss), especially since you should report crossentropy loss.
* Q: When I pick X% of pixels/examples, does it have to be exactly X% or can it be X% in expectation?
  * A: It's fine either way.
* Q: When I randomize pixels, should I randomize them again each time a particular example is drawn (each training step/epoch) or only once before training?
  * A: Each training step/epoch.
* Q: When I randomize labels, should I randomize them again each time a particular example is drawn (each training step/epoch) or only once before training?
  * A: Only once before training.
* Q: What is the expected length of report/explanation?
  * A: There is no minimum/maximum, but between 5 (concise) and 20 sentences should be good. Don't forget about plots.
* Q: When I replace labels/pixels with random values, what random distribution should I use?
  * A: A distribution reasonably similar to the data. However, you don't need to match dataset's distribution exactly - approximation will be totally fine, especially if it's faster or easier to get.
* Q: Can I use something different than Colab/Jupyter Notebook? E.g. just Python files.
  * A: Yes, although notebook is encouraged; please include in you solution code and pdf.

In [ ]:
import random
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Model definition and training for task 1.
The code for model is very similiar to the one in lab 4. Train and test function take additional arguments (losstrain and (accuracy, avgloss) respectively) to help plot the cross-entropy etc.

In [ ]:
#pip install plotly --upgrade # for the graphs to draw correctly, the plotly version should be updated to the newest, then restart runtime is required

In [ ]:
import plotly
plotly.__version__


'5.4.0'

In [ ]:
random.seed(24)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # After flattening an image of size 28x28 we have 784 inputs
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader, accuracy, avgloss):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy.append(correct/10000)
    test_loss /= len(test_loader.dataset)
    avgloss.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
batch_size = 256
test_batch_size = 1000
epochs = 5
lr = 1e-2
use_cuda = False
seed = 1
log_interval = 10

In [ ]:
use_cuda = not use_cuda and torch.cuda.is_available()

torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Below, I create empty lists to later fill them during training/testing to plot values.

In [ ]:
accuracy = []
accuracy0y = []
accuracy1y = []
accuracy2y = []
accuracy3y = []
accuracy4y = []

avgloss = []
avgloss0y = []
avgloss1y = []
avgloss2y = []
avgloss3y = []
avgloss4y = []

# Training models in setup 1: with randomized labels.

1. Label Distribution

At the graph below, we can see that the labels are rather equally distributed. So for example to randomize 20% of labels in task 1, I choose 20% of random values of list containing labels, and I replace them with random numbers from 0 to 9.

In [ ]:
labels = np.array(dataset1.targets)
value, count = np.unique(labels, return_counts=True)
df = pd.DataFrame(value)
df.columns = ['value']
df['count'] = count
fig = px.bar(df, x='value', y='count')
fig.update_layout(title={'text': '<b>Figure 1. Distribution of labels in data set MNIST</b>', 'x':0.5}, 
                  xaxis = dict(title = "label", tickmode= "linear"),
                  yaxis_title="Count")
fig.show()

In [ ]:
#X = 0%
dataset1 = datasets.MNIST('../data', train=True, download=False,
                    transform=transform)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader, accuracy, avgloss)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313259
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.732760
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.582762
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.481087
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.334523
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.346265
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.229349
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.191099
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.224889
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.244065
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.236049
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.241033
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.194208
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.302403
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.203934
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.192445
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.173173
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.140804
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.356528
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.130329
Train Epoc

In [ ]:
#X = 10%
dataset1 = datasets.MNIST('../data', train=True, download=False,
                    transform=transform)
Y = dataset1.targets
R = random.sample(range(1, 60000), 6000)

for i in R:
  Y[i] = random.randint(0,9)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader, accuracy0y, avgloss0y)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300735
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.995557
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.050178
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.819651
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.928061
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.038302
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.629039
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.657438
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.729345
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.747395
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.763967
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.746687
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.654411
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.671822
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.726343
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.036511
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.706761
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.752653
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.774402
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.800521
Train Epoc

In [ ]:
#X = 20%
dataset1 = datasets.MNIST('../data', train=True, download=False,
                    transform=transform)
Y = dataset1.targets
R = random.sample(range(1, 60000), 12000)

for i in R:
  Y[i] = random.randint(0,9)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader, accuracy1y, avgloss1y)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302898
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.487056
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.231255
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.156188
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.198650
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.121501
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.062418
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.029140
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.143948
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.303860
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.960363
Train Epoch: 1 [28160/60000 (47%)]	Loss: 1.076712
Train Epoch: 1 [30720/60000 (51%)]	Loss: 1.033062
Train Epoch: 1 [33280/60000 (55%)]	Loss: 1.127781
Train Epoch: 1 [35840/60000 (60%)]	Loss: 1.143099
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.031620
Train Epoch: 1 [40960/60000 (68%)]	Loss: 1.040429
Train Epoch: 1 [43520/60000 (72%)]	Loss: 1.058779
Train Epoch: 1 [46080/60000 (77%)]	Loss: 1.212402
Train Epoch: 1 [48640/60000 (81%)]	Loss: 1.005827
Train Epoc

In [ ]:
#X = 30%
dataset1 = datasets.MNIST('../data', train=True, download=False,
                    transform=transform)
Y = dataset1.targets
R = random.sample(range(1, 60000), 18000) 

for i in R:
  Y[i] = random.randint(0,9)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader, accuracy2y, avgloss2y)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311902
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.619639
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.452179
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.566507
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.478249
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.627117
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.492420
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.416904
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.513885
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.395811
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.296685
Train Epoch: 1 [28160/60000 (47%)]	Loss: 1.368141
Train Epoch: 1 [30720/60000 (51%)]	Loss: 1.435007
Train Epoch: 1 [33280/60000 (55%)]	Loss: 1.466619
Train Epoch: 1 [35840/60000 (60%)]	Loss: 1.535671
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.288364
Train Epoch: 1 [40960/60000 (68%)]	Loss: 1.523810
Train Epoch: 1 [43520/60000 (72%)]	Loss: 1.333440
Train Epoch: 1 [46080/60000 (77%)]	Loss: 1.391359
Train Epoch: 1 [48640/60000 (81%)]	Loss: 1.237626
Train Epoc

In [ ]:
#X = 60%
dataset1 = datasets.MNIST('../data', train=True, download=False,
                    transform=transform)
Y = dataset1.targets
R = random.sample(range(1, 60000), 36000) 

for i in R:
  Y[i] = random.randint(0,9)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader, accuracy3y, avgloss3y)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308616
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.188448
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.004684
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.007133
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.044257
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.080309
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.160310
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.929582
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.059110
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.097264
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.010744
Train Epoch: 1 [28160/60000 (47%)]	Loss: 1.960136
Train Epoch: 1 [30720/60000 (51%)]	Loss: 2.053653
Train Epoch: 1 [33280/60000 (55%)]	Loss: 1.950368
Train Epoch: 1 [35840/60000 (60%)]	Loss: 1.992306
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.963204
Train Epoch: 1 [40960/60000 (68%)]	Loss: 1.924479
Train Epoch: 1 [43520/60000 (72%)]	Loss: 2.013157
Train Epoch: 1 [46080/60000 (77%)]	Loss: 2.023723
Train Epoch: 1 [48640/60000 (81%)]	Loss: 2.000078
Train Epoc

In [ ]:
#X = 90%

dataset1 = datasets.MNIST('../data', train=True, download=False,
                    transform=transform)
Y = dataset1.targets
R = random.sample(range(1, 60000), 54000)

for i in R:
  Y[i] = random.randint(0,9)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader, accuracy4y, avgloss4y)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302938
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.334743
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.301679
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.313606
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.294082
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.298441
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.290937
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.295495
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.309052
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.287657
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.295591
Train Epoch: 1 [28160/60000 (47%)]	Loss: 2.295419
Train Epoch: 1 [30720/60000 (51%)]	Loss: 2.285267
Train Epoch: 1 [33280/60000 (55%)]	Loss: 2.271363
Train Epoch: 1 [35840/60000 (60%)]	Loss: 2.305431
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.297818
Train Epoch: 1 [40960/60000 (68%)]	Loss: 2.286381
Train Epoch: 1 [43520/60000 (72%)]	Loss: 2.299655
Train Epoch: 1 [46080/60000 (77%)]	Loss: 2.306534
Train Epoch: 1 [48640/60000 (81%)]	Loss: 2.296878
Train Epoc

# Training models in setup 2: with randomized pixels.

1. Distribution of pixel values in dataset.
For simplicity, I assume that distrubution of pixel values is the same for every image.
From the graph below, we see that approximately 80% of pixel values are 0 (or -0.4242 after scaling and normalization), and about 20% of values are let's say 255 (or 2.1825).

$$P(X = 0) = 0.8$$ $$P(X = 255) = 0.2 $$


In [ ]:
X = dataset1.train_data[4]
X = np.array(X)
X = X.reshape(784,1)
value, count = np.unique(X, return_counts=True)
df = pd.DataFrame(value)
df.columns = ['value']
df['count'] = count/784
fig = px.bar(df, x='value', y='count')
fig.update_layout(title={'text': '<b>Figure 2. Distribution of pixel values in dataset MNIST</b>', 'x':0.5}, 
                  xaxis_title="label",
                  yaxis_title="Count")
fig.show()

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:62: UserWarning:

train_data has been renamed data



Below I created empty lists to fill them later with values and then plot.

In [ ]:
accuracy0x = []
accuracy1x = []
accuracy2x = []
accuracy3x = []
accuracy4x = []

avgloss0x = []
avgloss1x = []
avgloss2x = []
avgloss3x = []
avgloss4x = []

In [ ]:
x_s = range(0,28)
y_s = range(0,28)
coord = set()
for i in x_s:
  for j in y_s:
    coord.add((i,j))
def train(model, device, train_loader, optimizer, epoch, log_interval, x_percent):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    target = target.to(device)
    data1 = data.detach().numpy()
    for i in range(data.shape[0]): 
      rand_coord = random.sample(coord, x_percent)
      rand_coord = np.array(rand_coord)
      x_tuple = tuple(rand_coord[:,0])
      y_tuple = tuple(rand_coord[:,1])
      data1[i][0][x_tuple,y_tuple] = np.random.choice([-0.4242, 2.8215], p = [0.8, 0.2], size = x_percent)
    data1 = torch.from_numpy(data1)
    data1 = data1.to(device)
    optimizer.zero_grad()
    output = model(data1)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
         print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [ ]:
#X ~ 10%
x_percent = 78
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval, x_percent)
    test(model, device, test_loader, accuracy0x, avgloss0x)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302832
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.819427
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.545395
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.444045
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.273252
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.286682
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.278621
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.198849
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.231043
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.285823
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.343673
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.295127
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.204936
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.252784
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.286947
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.247825
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.221061
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.165874
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.251044
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.136479
Train Epoc

In [ ]:
#X ~ 20%
x_percent = 156
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval, x_percent)
    test(model, device, test_loader, accuracy1x, avgloss1x)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.314288
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.663772
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.521297
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.501625
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.328393
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.405791
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.342608
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.237280
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.346289
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.297885
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.269918
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.285079
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.271893
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.246519
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.365915
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.247795
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.235313
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.222025
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.378923
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.164047
Train Epoc

In [ ]:
#X ~ 30%
model = Net().to(device)

x_percent = 235
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval, x_percent)
    test(model, device, test_loader, accuracy2x, avgloss2x)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320930
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.756417
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.640469
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.628328
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.470385
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.522021
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.328505
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.269200
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.390069
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.357527
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.355964
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.355478
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.356534
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.300664
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.378448
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.339488
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.291557
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.227414
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.475776
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.246190
Train Epoc

In [ ]:
#X = 60%
model = Net().to(device)
x_percent = 470
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval, x_percent)
    test(model, device, test_loader, accuracy3x, avgloss3x)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304724
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.423532
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.166179
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.175656
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.860574
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.064984
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.891352
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.703529
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.831849
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.837983
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.821499
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.860734
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.841192
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.749807
Train Epoch: 1 [35840/60000 (60%)]	Loss: 1.033985
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.790722
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.880304
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.639982
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.870015
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.728544
Train Epoc

In [ ]:
#X = 90%
model = Net().to(device)
x_percent = 705
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval, x_percent)
    test(model, device, test_loader, accuracy4x, avgloss4x)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311033
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.304622
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.216957
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.210023
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.166587
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.173583
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.171430
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.175526
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.145696
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.088882
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.075304
Train Epoch: 1 [28160/60000 (47%)]	Loss: 2.143808
Train Epoch: 1 [30720/60000 (51%)]	Loss: 2.218571
Train Epoch: 1 [33280/60000 (55%)]	Loss: 2.130688
Train Epoch: 1 [35840/60000 (60%)]	Loss: 2.242858
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.118967
Train Epoch: 1 [40960/60000 (68%)]	Loss: 2.123803
Train Epoch: 1 [43520/60000 (72%)]	Loss: 2.150966
Train Epoch: 1 [46080/60000 (77%)]	Loss: 2.129139
Train Epoch: 1 [48640/60000 (81%)]	Loss: 2.108483
Train Epoc

# Plots and report.

In [ ]:
#Setup 1 acc vs avg

fig = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy),
    go.Bar(name='avg loss', y=avgloss)
])
fig.update_layout(barmode='group', title={'text': '<b>Figure 4. Comparison of accuracy and average loss after each test epoch for X = 0%</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

fig0 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy0y),
    go.Bar(name='avg loss', y=avgloss0y)
])
fig0.update_layout(barmode='group', title={'text': '<b>Figure 5. Comparison of accuracy and average loss after each test epoch for X = 10% (only setup 1) </b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

fig1 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy1y),
    go.Bar(name='avg loss', y=avgloss1y)
])
# Change the bar mode
fig1.update_layout(barmode='group', title={'text': '<b>Figure 6. Comparison of accuracy and average loss after each test epoch for X = 20% (only setup 1)</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")
#plot 2 acc vs loss

fig2 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy2y),
    go.Bar(name='avg loss', y=avgloss2y)
])
# Change the bar mode
fig2.update_layout(barmode='group', title={'text': '<b>Figure 7. Comparison of accuracy and average loss after each test epoch for X = 30% (only setup 1)</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")
#plot 3 acc vs loss

fig3 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy3y),
    go.Bar(name='avg loss', y=avgloss3y)
])
# Change the bar mode
fig3.update_layout(barmode='group', title={'text': '<b>Figure 8. Comparison of accuracy and average loss after each test epoch for X = 60% (only setup 1)</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

#plot 4 acc vs loss
fig4 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy4y),
    go.Bar(name='avg loss', y=avgloss4y)
])
# Change the bar mode
fig4.update_layout(barmode='group', title={'text': '<b>Figure 9. Comparison of accuracy and average loss after each test epoch for X = 90% (only setup 1)</b>', 'x':0.5}, 
                 xaxis_title="Epoch number",
                 yaxis_title="Value")
fig.show()
fig0.show()
fig1.show()
fig2.show()
fig3.show()
fig4.show()

# **Setup 1. Accuracy and loss after each epoch – observations:**


*  As we can see on the graphs, replacing no more than 60% of labels with random ones, does not result in a significant decrease in accuracy.

*  Replacing 90% of labels to random ones still resulted in high accuracy (about 50%). That is very good compared to the model which assigns labels randomly (we would expect 10% of correct labels).

*  In most cases, accuracy and loss increase and decrease accordingly after each epoch – it means that model is learning.


In [ ]:
# TODO
#Setup 2, acc vs avg
fig = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy),
    go.Bar(name='avg loss', y=avgloss)
])
fig.update_layout(barmode='group', title={'text': '<b>Figure 10. Comparison of accuracy and average loss after each test epoch for X = 0%</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

fig0 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy0x),
    go.Bar(name='avg loss', y=avgloss0x)
])
fig0.update_layout(barmode='group', title={'text': '<b>Figure 11. Comparison of accuracy and average loss after each test epoch for X = 10% (only setup 2)</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")


fig1 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy1x),
    go.Bar(name='avg loss', y=avgloss1x)
])
# Change the bar mode
fig1.update_layout(barmode='group', title={'text': '<b>Figure 12. Comparison of accuracy and average loss after each test epoch for X = 20% (only setup 2)</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")
#plot 2 acc vs loss

fig2 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy2x),
    go.Bar(name='avg loss', y=avgloss2x)
])
# Change the bar mode
fig2.update_layout(barmode='group', title={'text': '<b>Figure 13. Comparison of accuracy and average loss after each test epoch for X = 30% (only setup 2)</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")
#plot 3 acc vs loss

fig3 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy3x),
    go.Bar(name='avg loss', y=avgloss3x)
])
# Change the bar mode
fig3.update_layout(barmode='group', title={'text': '<b>Figure 14. Comparison of accuracy and average loss after each test epoch for X = 60% (only setup 2)</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

#plot 4 acc vs loss
fig4 = go.Figure(data=[
    go.Bar(name='accuracy', y=accuracy4x),
    go.Bar(name='avg loss', y=avgloss4x)
])
# Change the bar mode
fig4.update_layout(barmode='group', title={'text': '<b>Figure 15. Comparison of accuracy and average loss after each test epoch for X = 90% (only setup 2)</b>', 'x':0.5}, 
                 xaxis_title="Epoch number",
                 yaxis_title="Value")
fig.show()
fig0.show()
fig1.show()
fig2.show()
fig3.show()
fig4.show()

# **Setup 2. Accuracy and loss after each epoch – observations:**

**We come to similar conclusions as in the first configuration:**

*  Replacing no more 60% than of pixels to random ones, does not result in a significant decrease in accuracy.

*  Replacing 90% of pixels to random ones still resulted in high accuracy (about 65%). That is still very good compared to the model which assigns labels randomly (we would expect 10% of correct labels).

*  In most cases, accuracy and loss increase and decreases accordingly after each epoch – it means that model is learning.

*  **replacing 10% of random pixels increased accuracy of about 0.5 percentage point compared to the original model where no replacement of pixels was done**

In [ ]:
# TODO
#both setups acc
fig0 = go.Figure(data=[
    go.Bar(name='accuracy (setup 1)', y=accuracy0y),
    go.Bar(name='accuracy (setup 2)', y=accuracy0x)
])
# Change the bar mode
fig0.update_layout(barmode='group', title={'text': '<b>Figure 16. Comparison of accuracy after each test epoch for X = 10% for both setups</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

fig1 = go.Figure(data=[
    go.Bar(name='accuracy (setup 1)', y=accuracy1y),
    go.Bar(name='accuracy (setup 2)', y=accuracy1x)
])
# Change the bar mode
fig1.update_layout(barmode='group', title={'text': '<b>Figure 17. Comparison of accuracy after each test epoch for X = 20% for both setups</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")
#plot 2 acc vs loss

fig2 = go.Figure(data=[
    go.Bar(name='accuracy (setup 1)', y=accuracy2y),
    go.Bar(name='accuracy (setup 2)', y=accuracy2x)
])
# Change the bar mode
fig2.update_layout(barmode='group', title={'text': '<b>Figure 18. Comparison of accuracy after each test epoch for X = 30% for both setups</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")
#plot 3 acc vs loss

fig3 = go.Figure(data=[
    go.Bar(name='accuracy (setup 1)', y=accuracy3y),
    go.Bar(name='accuracy (setup 2)', y=accuracy3x)
])
# Change the bar mode
fig3.update_layout(barmode='group', title={'text': '<b>Figure 19. Comparison of accuracy after each test epoch for X = 60% for both setups</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

#plot 4 acc vs loss
fig4 = go.Figure(data=[
    go.Bar(name='accuracy (setup 1)', y=accuracy4y),
    go.Bar(name='accuracy (setup 2)', y=accuracy4x)
])
# Change the bar mode
fig4.update_layout(barmode='group', title={'text': '<b>Figure 20. Comparison of accuracy after each test epoch for X = 90% for both setups</b>', 'x':0.5}, 
                 xaxis_title="Epoch number",
                 yaxis_title="Value")
fig1.show()
fig1.show()
fig2.show()
fig3.show()
fig4.show()

# **Comparison of accuracies for setup 1 and 2 for diffrent X% – observations:**




* for setup 2, in each epoch, accuracies are **only slightly higher** for different X%, than accuracies for setup 1,
* the biggest difference in accuracy we observe for X = 90% (in favor of setup 2).




In [ ]:
# TODO
#both setups avg loss

fig0 = go.Figure(data=[
    go.Bar(name='average loss (setup 1)', y=avgloss0y),
    go.Bar(name='average loss (setup 2)', y=avgloss0x)
])
# Change the bar mode
fig0.update_layout(barmode='group', title={'text': '<b>Figure 21. Comparison of average loss after each test epoch for X = 10% for both setups</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

fig1 = go.Figure(data=[
    go.Bar(name='average loss (setup 1)', y=avgloss1y),
    go.Bar(name='average loss (setup 2)', y=avgloss1x)
])
# Change the bar mode
fig1.update_layout(barmode='group', title={'text': '<b>Figure 22. Comparison of average loss after each test epoch for X = 20% for both setups</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")
#plot 2 acc vs loss

fig2 = go.Figure(data=[
    go.Bar(name='average loss (setup 1)', y=avgloss2y),
    go.Bar(name='average loss (setup 2)', y=avgloss2x)
])
# Change the bar mode
fig2.update_layout(barmode='group', title={'text': '<b>Figure 23. Comparison of average loss after each test epoch for X = 30% for both setups</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")
#plot 3 acc vs loss

fig3 = go.Figure(data=[
    go.Bar(name='average loss (setup 1)', y=avgloss3y),
    go.Bar(name='average loss (setup 2)', y=avgloss3x)
])
# Change the bar mode
fig3.update_layout(barmode='group', title={'text': '<b>Figure 24. Comparison of average loss after each test epoch for X = 60% for both setups</b>', 'x':0.5}, 
                  xaxis_title="Epoch number",
                  yaxis_title="Value")

#plot 4 acc vs loss
fig4 = go.Figure(data=[
    go.Bar(name='average loss (setup 1)', y=avgloss4y),
    go.Bar(name='average loss (setup 2)', y=avgloss4x)
])
# Change the bar mode
fig4.update_layout(barmode='group', title={'text': '<b>Figure 25. Comparison of average loss after each test epoch for X = 90% for both setups</b>', 'x':0.5}, 
                 xaxis_title="Epoch number",
                 yaxis_title="Value")
fig0.show()
fig1.show()
fig2.show()
fig3.show()
fig4.show()

# **Comparison of losses for setup 1 and 2 for diffrent X% – observations:**



*  for setup 2, in each epoch, **loss is significantly** lower for different X%, than for setup 1

In [ ]:
#setup 1
accurcies_y = [accuracy[4], accuracy0y[4], accuracy1y[4], accuracy2y[4], accuracy3y[4], accuracy4y[4]]

avg_losses_y = [avgloss[4], avgloss0y[4], avgloss1y[4], avgloss2y[4], avgloss3y[4], avgloss4y[4]]

#setup 2
accurcies_x = [accuracy[4], accuracy0x[4], accuracy1x[4], accuracy2x[4], accuracy3x[4], accuracy4x[4]]

avg_losses_x = [avgloss[4], avgloss0x[4], avgloss1x[4], avgloss2x[4], avgloss3x[4], avgloss4x[4]]

In [ ]:
# TODO 
#avg loss for difrent X


fig1 = go.Figure(data=[
    go.Bar(name='accuracy after 5th epoch (setup 1)', y=accurcies_y),
    go.Bar(name='average loss after 5th epoch (setup 1)', y=avg_losses_y)
])
# Change the bar mode
fig1.update_layout(barmode='group', title={'text': '<b>Figure 26. Comparison of accuracy and average loss after 5th epoch for diffrent X values (setup 1)</b>', 'x':0.5}, 
                  xaxis = dict(title = "Percent of randomized labels",tickvals=[0,1,2,3,4,5,6], ticktext = ['0%', '10%', '20%', '30%', '60%', '90%']),
                  yaxis_title="Value")
#plot 2 acc vs loss

fig2 = go.Figure(data=[
    go.Bar(name='accuracy after 5th epoch (setup 2)', y=accurcies_x),
    go.Bar(name='average loss after 5th epoch (setup 2)', y=avg_losses_x)
])
# Change the bar mode
fig2.update_layout(barmode='group', title={'text': '<b>Figure 27. Comparison of accuracy and average loss after 5th epoch for diffrent X values (setup 2)</b>', 'x':0.5}, 
                  xaxis = dict(title = "Percent of randomized pixels",tickvals=[0,1,2,3,4,5,6], ticktext = ['0%', '10%', '20%', '30%', '60%', '90%']),
                  yaxis_title="Value")
fig1.show()
fig2.show()

# **Observations**

* loss is growing faster in setup 1
* decrease in accuracy is higher for setup 1
* the biggest difference in loss is in the case where X = 90% – setup 2 achieves much lower loss.


# **Why does adding noise to labels does not result in improvement of accuracy and loss?**

Because when we add noise to labels, the model is just getting more false hopes of learning. However, if we add noise to input data, the model is learning on harder examples and it may – depending on noise – result in an increase of accuracy on unseen data. 

# **Why is accuracy still high even after we change data to 60% noise?**

The model is learning patterns, and it pretty much ignores the 60% of noisy data, because there are no patterns, and focuses only on that 40% where patterns are visible. However, if we if set noise to 90%, the remaining 10% of quality data is too little for the model to learn.

# **Summary**


*   adding some noise (in this case replacing 20% of pixels to random ones) to input data may result in an increase of accuracy on unseen data – we make our model more flexible (we reduce overfitting)

*   adding noise to labels does not result in increased accuracy on unseen data.

*   exchanging pixels/labels can take very long for huge datasets.

*   we may interpret adding noise to input data as a form of regularisation to reduce overfitting



